In [ ]:
import re
import time
import json
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options


def random_sleep():
    # 1부터 10 사이의 랜덤한 숫자 생성
    sleep_time = random.randint(1, 10)
    
    # 생성된 숫자를 출력
    # print(f"Sleeping for {sleep_time} seconds...")
    
    # sleep 함수를 사용하여 대기
    time.sleep(sleep_time)

    # 함수 호출 : random_sleep()





chrome_options = Options()
chrome_options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
driver = webdriver.Chrome(r"C:\chrome_driver\chromedriver.exe", options=chrome_options)

code_list = pd.read_csv("카테고리별_상품번호(전체).csv")
code = list(code_list['상품번호'])



result = []

for j in range(len(code)):
    print()
    print(j)
    a = code[j].replace("'",'"')
    category = list(code_list['카테고리'])[j]
    code_result = json.loads(a)
    
    for i in code_result:
        print()
        print(i)
        url = f"https://www.wadiz.kr/web/campaign/detail/{i}"
        review_url = f"https://www.wadiz.kr/web/campaign/detail/qa/{i}"  # 리뷰수 크롤링용 주소
        
        driver.get(url)
        # time.sleep(1.5)
        time.sleep(5)
        random_sleep()
        
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 달성 비율
        # time.sleep(2)
        random_sleep()
        goal_per = soup.find_all("span",{"class":"Badge_badge__ovUKI Badge_label__2Rft2 Badge_md__1Ck0w Badge_secondary__UV-TU Badge_tertiary__3uYKh"})[0].get_text()
        goal_per = float(re.findall("\d+",str(goal_per))[0])
        print(" - 달성비율 : ", goal_per)

        # 최종 금액
        # final_price = driver.find_element(By.XPATH,'//*[@id="main-app"]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[3]/div[2]/p').text
        # final_price = soup.find_all("span",{"class":"Badge_badge__ovUKI Badge_label__2Rft2 Badge_md__1Ck0w Badge_secondary__UV-TU Badge_tertiary__3uYKh"})[0].get_text()
        # time.sleep(2)
        random_sleep()
        final_price = driver.find_element(By.XPATH,'//*[@id="main-app"]/div[2]/div/div[2]/div/div[2]/div/div[1]/div[4]/div[2]/p').text
        final_price = float(re.findall("\d+",str(final_price))[0])
        print(" - 최종금액 : ", final_price)

        # 목표금액
        # time.sleep(2)
        random_sleep()
        if final_price == 0 or goal_per == 0:
            goal_price = 0
        else:
            goal_price = final_price / (goal_per/100)
        print(" - 목표금액 : ", goal_price)

        # 서포터 수
        # Supporter = soup.find_all("span",{"Tabs_count__1atno"})[-1].get_text()
        # Supporter = soup.find_all("span",{"class":"SupportSignatureSignMobile_buttonBox__YYAfj SupportSignatureSignMobile_disabled__3-lDA"})[0].get_text()
        # time.sleep(2)
        random_sleep()
        try:
            span_element = soup.find('span', class_='MakerInfoBody_iconPerson__Ef5v0')
            supporter_count = span_element.next_sibling  # 다음 형제 요소를 가져옵니다.
            supporter_count_text = span_element.next_sibling.get_text(strip=True)
            Supporter = int(''.join(filter(str.isdigit, supporter_count_text)))
            Supporter = float(Supporter)
        except:
            Supporter = 0
        print(" - 서포터 수 : ", Supporter)

        # 찜한 개수
        # heart = soup.find_all("span",{"RewardCTA_count__3Bg5y"})[0].get_text()
        # heart = soup.find_all("span",{"class":"Button_button__2FuOU Button_secondary__LNLsN Button_lg__1Dftp Button_block__1-g8w RewardCTA_wishButton__3JmIg"})[0].get_text()
        # time.sleep(2)
        random_sleep()
        try:
            heart = soup.find_all("span",{"RewardCTA_count__3Bg5y"})[0].get_text()
            heart = heart.replace(",","")
            heart = float(heart)
        except:
            heart = 0
        print(" - 찜 : ", heart)

        # 지지서명 개수
        # time.sleep(2)
        random_sleep()
        try:
            Applause = soup.find_all("span",{"SupportSignatureSignMobile_count__1bbSa"})[0].get_text()
            Applause = float(Applause)
        except:
            Applause = 0
        print(" - 지지서명 : ", Applause)

        # 진행기간
        # time.sleep(2)
        random_sleep()
        year = soup.find_all("div",{"MessageBox_description__2Fyxq"})[-1]
        year = re.findall("\d+",str(year))[1]

        Period = soup.find_all("div", {"Rewards_period__Mo1m-"})[-1].get_text()
        match = re.search(r"진행기간(\d+\.\d+) ~ (\d+\.\d+)", Period)
        
        if match:
            period_start = match.group(1).replace(".", "")
            period_end = match.group(2).replace(".", "")
            Period = [period_start, period_end]
        Period = [year + Period[0], year + Period[1]]
        
        # 펀딩 시작일
        start_fun = datetime.strptime(Period[0], "%Y%m%d")

        # 펀딩 완료일
        end_fun = datetime.strptime(Period[1], "%Y%m%d")

        # 펀딩 기간
        Period = datetime.strptime(Period[1], "%Y%m%d") - datetime.strptime(Period[0], "%Y%m%d")
        print(" - 펀딩기간 : ", Period)

        # 배송까지 기간
        # time.sleep(2)
        random_sleep()
        wait = WebDriverWait(driver, 30)          # 최대 30초간 대기
        try:
            Period_Ship = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-app"]/div[2]/div/div[2]/div/div[2]/div/div[5]/div[1]/div[2]/div/div/div/div[3]/div/div/div[4]/span[2]'))).text
            Period_Ship = re.findall("\d+",str(Period_Ship))
            Period_Ship = datetime.strptime(Period_Ship[0] + Period_Ship[1] + Period_Ship[2],"%Y%m%d") - end_fun
        
        except:
            # Period_Ship = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="main-app"]/div[2]/div/div[2]/div/div[2]/div/div[6]/div[1]/div[2]/div/div[1]/div/div[3]/div/div/div[4]/span[2]'))).text
            Period_Ship = 99 # 기타 : 배송기간이 없는 경우 99로 처리
            
        print(" - 배송까지 기간 : ", Period_Ship)

        # 회사
        #time.sleep(2)
        random_sleep()
        try:
            company = soup.find_all("span",{"MakerInfoHeader_makerName__jLMZK"})[0].get_text()
        except:
            company = 0
        print(" - 회사명 : ", company)

        # 회사 팔로워
        # time.sleep(2)
        random_sleep()
        try:
            company_follower = soup.find_all("span",{"MakerInfoHeader_subInfo__1JZ1L"})[0].get_text()
            company_follower = re.findall("\d+",str(company_follower))[0]
        except:
            company_follower = 0
        print(" - 회사 팔로워 : ", company_follower)


        #사진 개수
        # time.sleep(2)
        random_sleep()
        try:
            photo_number = soup.find_all("span",{"RewardImageSlider_slickIndex__1OeOW"})[0].get_text()
            photo_number = re.findall("\d+",photo_number)[-1]
            photo_number = float(photo_number)
            # photo_movie = 1
            photo_movie = 0
        except:
            photo_number = 1
            photo_movie = 0
        print(" - 이미지 수 : ", photo_number)
        print(" - 동영상 수 : N/A")


        # 펀딩종류(펀딩 / 프리오더)
        # time.sleep(2)
        random_sleep()
        try:
            kind = soup.find_all("span",{"NoticeBanner_title__FjAI9"})[0].get_text()
        except:
            kind = 0
        print(" - 펀딩종류 : ", kind)





        # 리뷰 수 (응원·의견·체험)
        driver.get(review_url)
        time.sleep(5)
        random_sleep()
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # time.sleep(2)
        random_sleep()
        try:
            review = soup.find_all("span",{"CommonCommentTitle_total__aWnLU"})[0].get_text()
            review = float(review)
        except:
            review = 0
        print(" - 리뷰 수 : ", review)
        
        # 결과 append
        result.append([category,i,goal_per,final_price,goal_price,Supporter,heart,Applause,start_fun,end_fun,Period,Period_Ship,company,company_follower,photo_number,photo_movie,kind])
        print(" ## 리스트 추가 완료")


# 데이터프레임 생성
print(" ## 데이터프레임 생성중")
df = pd.DataFrame(result,columns=['category','product_code','goal_per','final_price','goal_price','Supporter','heart','Applause','start_fun','end_fun','Period','Period_Ship','company','company_follower','photo_number','photo_movie','kind'])


# CSV 파일로 저장
print(" ## CSV 파일 생성중")
df.to_csv('와디즈 변수 크롤링(이미지 제외).csv')
print("complete!")

